# Plane Wave

## Simulation of Physical Cavity Setup

![](images/cavity.png)

* the length of the `cavityspace` is dependent on the focal length of the mirrors
    * in our case the focal length is 10cm so the `cavityspace` will be 20cm
* whether the cavity is in resonance depends on the the exact distance between the two mirrors
    * if it's a multiple of the `laser`'s frequency the cavity will be in resonance
    * in Finesse2 this is represented by the **tuning** (phi) of a mirror

---

Small distance adjustments (nanometers) should be accomplished via changing the tuning of a component, large adjustments via the length parameter of a space.
**Small adjustments will NOT work via the length parameter!**

### High Reflectivity, phi sweep

* `piezomirror` R of 99,7%
* `lambda` for phi is set in [kat.ini](kat.ini) (in our case: `lambda 6.33E-7`, 633 nanometers)
* `piezomirror`: a positive phi moves the mirror `node4` closer to `node3`

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *

kat = finesse.kat()

reflectivity = 0.997
code = f"""
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror {reflectivity} {1-reflectivity} 0 node3 node4
pd photodiode node4

xaxis piezomirror phi lin 0 360 10_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)

### High Reflectivity with modeled mirrors, phi sweep

* `piezomirror` R of with modeled mirrors 99,7%
* `lambda` for phi is set in [kat.ini](kat.ini) (in our case: `lambda 6.33E-7`, 633 nanometers)
* `piezomirror`: a positive phi moves the mirror `node4` closer to `node3`

* links:
    * doku Finesse 2: http://www.gwoptics.org/finesse/download/manual.pdf
    * mirror: https://finesse.docs.ligo.org/finesse3/usage/elements/optics/mirror.html?highlight=mirror#kat-element-mirror
        * 632,816 nm wavelength He-Ne-laser
        * transmission
        * inner reflectivity: 0,997
        * focal length: 10cm => radius: 20cm 
    * space:
        * Refractive Index (adjust!) NBK-7: https://refractiveindex.info/?shelf=glass&book=BK7&page=SCHOTT
        * ==> substrate of our mirror: 1.5151 refractive index

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *

kat = finesse.kat()

reflectivity = 0.997
transmittance = 0.5

# FINESSE 2 !!!
# MIRROR:  m name Reflectivity Transmittance Tuning NODES
# SPACE: s name portA=none portB=none LENGTH=0.0 REFRACTIV_INDEX=1.0 user_gouy_x=none user_gouy_y=none

code = f"""
l laser 1.2M 0 node0
pd pre_cavity1 node0

s prespace 0.1 node0 node1
pd pre_cavity2 node1

m fixedOuter 0.003 0.997 0 node1 node2
s fixedSpace 0.006 1.5151 node2 node3
m fixedInner 0.997 0.003 0 node3 node4
attr fixedInner Rc 0.2

s cavityspace 0.2 node4 node5
pd pre_piezo node5

m piezoInner 0.997 0.003 0 node5 node6
attr piezoInner Rc 0.2
s piezoSpace 0.006 1.5151 node6 node7
m piezoOuter 0.003 0.997 0 node7 node8

pd post_cavity node8
xaxis piezoInner phi lin 0 360 10_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)

### Low Reflectivity, phi sweep


* `piezomirror` R of 90%
* a lower reflectivity reduces the power output of the cavity

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *
import matplotlib.pyplot as plt

kat = finesse.kat()
reflectivity = 0.9

code = f"""
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror {reflectivity} {1-reflectivity} 0 node3 node4
pd photodiode node4

xaxis piezomirror phi lin 0 360 10_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)

In [ ]:
from IPython.display import display
import ipywidgets as widgets

switch = True

button = widgets.Button(description="toggle scaling")
output = widgets.Output()

display(button, output)


def on_button_clicked(b):
    global switch
    if switch:
        plt.ylim(top=1259999.864594087)
    else:
        plt.ylim(top=135888.67754189012)
    switch = not switch


button.on_click(on_button_clicked)

### Reflectivity with phi==0, R sweep

When **phi** is zero degress the cavity is in perfect resonance.

* phi is fixed to zero
* the reflectivity of `piezomirror` is swept from 90% to 100%
* 1.000 steps
* higher reflectivity leads to exponentially more cavity power output

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror .997 .003 0 node3 node4
pd photodiode node4

xaxis piezomirror R lin .9 1 1_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)
print(kat.laser.P)

### Laser power at phi==0, P sweep

* phi is fixed to zero
* the power output of `laser` is swept from 0 to 3 Megawatts
* higher power input leads to a linear increase of cavity power output
* 1.000 steps

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror .997 .003 0 node3 node4
pd photodiode node4

xaxis laser P lin 0 3M 1_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
# fig.set_figheight(5)
# fig.set_figwidth(11)
print(kat.laser.P)

### labeling

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror 0.997 .003 0 node3 node4

pd refl node1          # Reflected field
pd circ node3          # Circulating field
pd tran node4          # Transmitted field

xaxis piezomirror phi lin -20 200 300
"""

kat.parse(code)
out = kat.run()
fig = out.plot(
    return_fig=True,
    title="Power vs. cavity length",
    xlabel="position of mirror [deg], phi",
    ylabel="Power output [W], P",
)
print(kat.laser.P)

# Gaussian Finesse Model

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact

pykat.init_pykat_plotting(dpi=90)
phi = widgets.IntSlider(
    min=0,
    max=360,
    value=10,
    description="phi",
    step=1,
    continuous_update=True,
    orientation="horizontal",
)


def f(change):
    code = f"""
    l laser 1 0 node0
    s prespace 0.1 node0 node1
    m fixedmirror 0.997 0.003 0 node1 node2
    s cavityspace 0.2 node2 node3
    m piezomirror 0.997 .003 {change.new} node3 node4

    beam beamprofile node4

    xaxis beamprofile x lin -4 4 30
    x2axis beamprofile y lin -4 4 30
    """

    print(code)


phi.observe(f, names="value")
display(phi)

kat = finesse.kat()

kat.parse(code)
out = kat.run()

plt.figure(figsize=(6, 6))
plt.grid(False)
plt.pcolormesh(out.x, out.y, out["beamprofile"], shading="auto")
print(out["beamprofile"][15][15])
print(phi.value)

# slider example, mpl_interactions

In [ ]:
%matplotlib ipympl
import mpl_interactions.ipyplot as iplt
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, np.pi, 100)
tau = np.linspace(0.5, 10, 100)


def f1(x, tau, beta):
    return np.sin(x * tau) * x * beta


def f2(x, tau, beta):
    return np.sin(x * beta) * x * tau


fig, ax = plt.subplots()
controls = iplt.plot(x, f1, tau=tau, beta=(1, 10, 100), label="f1")
iplt.plot(x, f2, controls=controls, label="f2")
_ = plt.legend()
plt.show()